In [9]:
import sys
sys.path.append('../unseen')

from dask.distributed import Client, LocalCluster
import xclim
import xarray as xr

import myfuncs
import indices

In [2]:
cluster = LocalCluster()
client = Client(cluster)

In [3]:
client

Client Scheduler: tcp://127.0.0.1:39683 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 28.00 GiB


In [4]:
history = myfuncs.get_new_log()
print(history)

Fri Jun 25 11:17:21 2021: /g/data/e14/dbi599/miniconda3/envs/unseen/bin/jupyter notebook /home/599/dbi599/unseen/notebooks/preprocess.ipynb


# CAFE

In [5]:
cafe_files = ['/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19911101/ZARR/atmos_isobaric_daily.zarr.zip',
              '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19921101/ZARR/atmos_isobaric_daily.zarr.zip']

In [6]:
cafe_kwargs = {'metadata_file': '../config/dataset_cafe.yml',
               'no_leap_days': True,
               'region': 'TAS-POINT',
               'variables': ['pr'],
               'units' : {'pr': 'mm/day'},
              }

In [7]:
cafe_outfile = '/g/data/xv83/dbi599/pr_cafe-c5-d60-pX-f6_19911101-19921101_3650D_cafe-grid-TAS-POINT.zarr.zip'

In [8]:
cafe_ds = myfuncs.open_mfforecast(cafe_files, **cafe_kwargs)
cafe_ds.attrs['history'] = history
cafe_ds

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 2, lead_time: 3650)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
  * init_date  (init_date) datetime64[ns] 1991-11-01 1992-11-01
    time       (lead_time, init_date) datetime64[ns] 1991-11-01 ... 2002-10-29
Data variables:
    pr         (init_date, lead_time, ensemble) float32 dask.array<chunksize=(1, 28, 96), meta=np.ndarray>
Attributes:
    comment:    pressure level interpolator, version 3.0, precision=double
    filename:   atmos_isobaric_daily.zarr
    grid_tile:  N/A
    grid_type:  regular
    title:      AccessOcean-AM2
    history:    Fri Jun 25 11:17:21 2021: /g/data/e14/dbi599/miniconda3/envs/...

In [9]:
cafe_ds['pr']

<xarray.DataArray 'pr' (init_date: 2, lead_time: 3650, ensemble: 96)>
dask.array<concatenate, shape=(2, 3650, 96), dtype=float32, chunksize=(1, 28, 96), chunktype=numpy.ndarray>
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
  * init_date  (init_date) datetime64[ns] 1991-11-01 1992-11-01
    time       (lead_time, init_date) datetime64[ns] 1991-11-01 ... 2002-10-29
Attributes:
    cell_methods:   time: mean
    interp_method:  conserve_order1
    long_name:      Total precipitation rate
    time_avg_info:  average_T1,average_T2,average_DT
    units:          kg/m2/s

In [19]:
cafe_ds = cafe_ds.chunk({'lead_time': 50})

In [20]:
cafe_ds['pr']

<xarray.DataArray 'pr' (init_date: 2, lead_time: 3650, ensemble: 96)>
dask.array<mul, shape=(2, 3650, 96), dtype=float32, chunksize=(1, 50, 96), chunktype=numpy.ndarray>
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
  * init_date  (init_date) datetime64[ns] 1991-11-01 1992-11-01
    time       (lead_time, init_date) datetime64[ns] dask.array<chunksize=(50, 2), meta=np.ndarray>
Attributes:
    cell_methods:   time: mean
    interp_method:  conserve_order1
    long_name:      Total precipitation rate
    time_avg_info:  average_T1,average_T2,average_DT
    units:          mm d-1

In [21]:
myfuncs.to_zarr(cafe_ds, cafe_outfile)

# AWAP

In [14]:
awap_infile = '/g/data/xv83/ds0092/data/csiro-dcfp-csiro-awap/rain_day_19000101-20201202_cafe-grid.zarr/'
awap_outfile = '/g/data/xv83/dbi599/pr_awap_cafe-grid-TAS-POINT.zarr.zip'

In [15]:
awap_kwargs = {'metadata_file': '../config/dataset_awap.yml',
               'no_leap_days': True,
               'region': 'TAS-POINT',
               'variables': ['pr'],
               'units' : {'pr': 'mm/day'},
              }

In [16]:
awap_ds = myfuncs.open_file(awap_infile, **awap_kwargs)
awap_ds.attrs['history'] = history
awap_ds

<xarray.Dataset>
Dimensions:  (time: 44136)
Coordinates:
  * time     (time) object 1900-01-01 09:00:00 ... 2020-12-02 09:00:00
Data variables:
    pr       (time) float64 dask.array<chunksize=(44136,), meta=np.ndarray>
Attributes:
    history:  Fri Jun 25 11:17:21 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [40]:
awap_ds.indexes['time']

CFTimeIndex([1900-01-01 09:00:00, 1900-01-02 09:00:00, 1900-01-03 09:00:00,
             1900-01-04 09:00:00, 1900-01-05 09:00:00, 1900-01-06 09:00:00,
             1900-01-07 09:00:00, 1900-01-08 09:00:00, 1900-01-09 09:00:00,
             1900-01-10 09:00:00,
             ...
             2020-11-23 09:00:00, 2020-11-24 09:00:00, 2020-11-25 09:00:00,
             2020-11-26 09:00:00, 2020-11-27 09:00:00, 2020-11-28 09:00:00,
             2020-11-29 09:00:00, 2020-11-30 09:00:00, 2020-12-01 09:00:00,
             2020-12-02 09:00:00],
            dtype='object',
            length=44136,
            calendar='proleptic_gregorian',
            freq='None')

In [29]:
dir(test)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__inv__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_accessors',
 '_arith_method',
 '_assert_can_do_setop',

In [25]:
print(xr.infer_freq(awap_ds.indexes['time'].to_datetimeindex()))

None


In [24]:
awap_ds.indexes['time'].to_datetimeindex()

DatetimeIndex(['1900-01-01 09:00:00', '1900-01-02 09:00:00',
               '1900-01-03 09:00:00', '1900-01-04 09:00:00',
               '1900-01-05 09:00:00', '1900-01-06 09:00:00',
               '1900-01-07 09:00:00', '1900-01-08 09:00:00',
               '1900-01-09 09:00:00', '1900-01-10 09:00:00',
               ...
               '2020-11-23 09:00:00', '2020-11-24 09:00:00',
               '2020-11-25 09:00:00', '2020-11-26 09:00:00',
               '2020-11-27 09:00:00', '2020-11-28 09:00:00',
               '2020-11-29 09:00:00', '2020-11-30 09:00:00',
               '2020-12-01 09:00:00', '2020-12-02 09:00:00'],
              dtype='datetime64[ns]', length=44136, freq=None)

In [8]:
awap_ds['pr']

<xarray.DataArray 'pr' (time: 44136)>
dask.array<getitem, shape=(44136,), dtype=float64, chunksize=(44136,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1900-01-01 09:00:00 ... 2020-12-02 09:00:00
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm d-1
    valid_range:                   [-1.0, 100000.0]

In [21]:
myfuncs.to_zarr(awap_ds, awap_outfile)